In [1]:
import numpy as np
import pandas as pd

In [159]:
data=pd.read_csv("./project/train.csv")
data["id"]=data["encounter_id"]
data.drop("encounter_id",axis=1,inplace=True)
data.drop("country",axis=1,inplace=True)
data.drop("patient_id",axis=1,inplace=True)
data.drop("average_pulse_bpm",axis=1,inplace=True)
data.drop("weight",axis=1,inplace=True)
data.set_index("id",inplace=True)

In [160]:
data=data[data["primary_diagnosis"]!="?"]

In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71220 entries, 533253 to 459757
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   race                                   67668 non-null  object
 1   gender                                 71220 non-null  object
 2   age                                    67663 non-null  object
 3   payer_code                             71220 non-null  object
 4   outpatient_visits_in_previous_year     71220 non-null  int64 
 5   emergency_visits_in_previous_year      71220 non-null  int64 
 6   inpatient_visits_in_previous_year      71220 non-null  int64 
 7   admission_type                         67515 non-null  object
 8   medical_specialty                      71220 non-null  object
 9   discharge_disposition                  68630 non-null  object
 10  admission_source                       66503 non-null  object
 11  length_of_stay

In [162]:
#data["weight"]=data["weight"].map(lambda x: 0 if x=="?" else 1)

In [163]:
race_map={
    "Caucasian":"cauc",
    "AfricanAmerican":"afam",
    "Hispanic":"other",
    "Asian":"other",
    "Other":"other",
    "?":"other",
    np.nan:"other"}

In [164]:
for cat,name in race_map.items():
    if "race_("+name+")" in data.columns:
        data["race_("+name+")"]|=(data["race"]==cat).astype("int64")
    else:
        data["race_("+name+")"]=(data["race"]==cat).astype("int64")
data.drop("race",axis=1,inplace=True)
data=data.copy()

In [165]:
data["gender"]=data["gender"].map(lambda g: 1 if g=="Female" else 0)

In [166]:
data["age"]=data["age"].map(lambda age: int(age[1]) if not age is np.nan else np.nan).astype("Int64")

In [167]:
payer_code_map={
    "?":"none",
    "MC":"mc",
    "HM":"hm",
    "SP":"sp",
    "BC":"bc",
    "MD":"md",
    "CP":"cp",
    "UN":"un",
    "CM":"cm",
    "OG":"other",
    "PO":"other",
    "DM":"other",
    "CH":"other",
    "WC":"other",
    "OT":"other",
    "MP":"other",
    "SI":"other",
    "FR":"other"}

In [168]:
for cat,name in payer_code_map.items():
    if "payer_code_("+name+")" in data.columns:
        data["payer_("+name+")"]|=(data["payer_code"]==cat).astype("int64")
    else:
        data["payer_("+name+")"]=(data["payer_code"]==cat).astype("int64")
data.drop("payer_code",axis=1,inplace=True)
data=data.copy()

In [169]:
data.rename({"outpatient_visits_in_previous_year":"#_outpatient",
    "emergency_visits_in_previous_year":"#_emergency",
    "inpatient_visits_in_previous_year":"#_inpatient"},
    axis=1,inplace=True)

In [170]:
data["#_of_visits"]=data["#_outpatient"]+data["#_inpatient"]+data["#_emergency"]

In [171]:
for column in {"#_of_visits","#_outpatient","#_inpatient","#_emergency"}:
    #data[column]=data[column].map(lambda x: 10 if x>10 else x)
    data[column]=data[column].map(lambda value: np.log1p(value))
    minimum=data[column].min()
    data[column]=(data[column]-minimum)/(data[column].max()-minimum)

In [172]:
admission_type_map={
    "Elective":"elec",
    "Emergency":"emerg",
    "Urgent":"urg",
    "Not Available":"other",
    "Not Mapped":"other",
    "Trauma Center":"other",
    "Newborn":"other",
    np.nan:"other"}

In [173]:
for cat,name in admission_type_map.items():
    if "adm_type_("+name+")" in data.columns:
        data["adm_type_("+name+")"]|=(data["admission_type"]==cat).astype("int64")
    else:
        data["adm_type_("+name+")"]=(data["admission_type"]==cat).astype("int64")
data.drop("admission_type",axis=1,inplace=True)
data=data.copy()

In [174]:
medical_specialty_map={
    'Trauma': 'emerg',
    'Emergency/Trauma': 'emerg',
    'Family/GeneralPractice': 'fam',
    'GeneralPractice': 'fam',
    'Surgery-General': 'surg',
    'Surgeon': 'surg',
    'Surgery-Cardiovascular': 'surg',
    'Surgery-Cardiovascular/Thoracic': 'surg',
    'Surgery-Colon&Rectal': 'surg',
    'Surgery-Maxillofacial': 'surg',
    'Surgery-Neuro': 'surg',
    'Surgery-Pediatric': 'surg',
    'Surgery-PlasticwithinHeadandNeck': 'surg',
    'Surgery-Plastic': 'surg',
    'Surgery-Thoracic': 'surg',
    'Surgery-Vascular': 'surg',
    'SurgicalSpecialty': 'surg',
    'Orthopedics': 'ortho',
    'Orthopedics-Reconstructive': 'ortho',
    'InternalMedicine': 'int',
    'Cardiology': 'cardio',
    'Endocrinology': 'endo',
    'Endocrinology-Metabolism': 'endo',
    'Gastroenterology': 'gastro',
    'Hematology': 'hema',
    'Hematology/Oncology': 'hema',
    'InfectiousDiseases': 'infec',
    'Nephrology': 'neph',
    'Oncology': 'onco',
    'Pulmonology': 'pulm',
    'Rheumatology': 'rheum',
    'Urology': 'uro',
    'Neurology': 'neuro',
    'Neurophysiology': 'neuro',
    'Dermatology': 'derma',
    'Gynecology': 'obgy',
    'Obsterics&Gynecology-GynecologicOnco': 'obgy',
    'Obstetrics': 'obgy',
    'ObstetricsandGynecology': 'obgy',
    'Psychiatry': 'psychi',
    'Psychology': 'psycho',
    'Pediatrics-AllergyandImmunology': 'pedia',
    'Pediatrics-CriticalCare': 'pedia',
    'Pediatrics-EmergencyMedicine': 'pedia',
    'Pediatrics-Endocrinology': 'pedia',
    'Pediatrics-Hematology-Oncology': 'pedia',
    'Pediatrics-InfectiousDiseases': 'pedia',
    'Pediatrics-Neurology': 'pedia',
    'Pediatrics-Pulmonology': 'pedia',
    'Cardiology-Pediatric': 'pedia',
    'Psychiatry-Child/Adolescent': 'pedia',
    'PhysicianNotFound': 'other',
    'Pediatrics':"pedia",
    'AllergyandImmunology':"imu",
    'Pathology':"path",
    'Proctology':"proc"}

In [175]:
for cat,name in medical_specialty_map.items():
    if "med_spec_("+name+")" in data.columns:
        data["med_spec_("+name+")"]|=(data["medical_specialty"]==cat).astype("int64")
    else:
        data["med_spec_("+name+")"]=(data["medical_specialty"]==cat).astype("int64")
data.drop("medical_specialty",axis=1,inplace=True)
data=data.copy()

In [176]:
discharge_disposition_map={
    #"Neonate discharged to another hospital for neonatal aftercare":"neo"
    "Discharged/transferred to SNF":"snf",
    "Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.":"snf",
    "Discharged/transferred to home with home health service":"home++",
    "Discharged/transferred to home under care of Home IV provider":"home+",
    "Discharged to home":"home",
    "Discharged/transferred to another rehab fac including rehab units of a hospital .":"rehab",
    "Discharged/transferred to another short term hospital":"short",
    "Discharged/transferred to another type of inpatient care institution":"long",
    "Discharged/transferred/referred to this institution for outpatient services":"out",
    "Discharged/transferred/referred another institution for outpatient services":"out",
    "Still patient or expected to return for outpatient services":"out",
    "Discharged/transferred to ICF":"icf",
    "Hospice / medical facility":"hospice",
    "Hospice / home":"hospice",
    "Expired at home. Medicaid only, hospice.":"hospice",
    "Discharged/transferred to a federal health care facility.":"long",
    "Discharged/transferred to a long term care hospital.":"long",
    "Discharged/transferred within this institution to Medicare approved swing bed":"long",
    "Admitted as an inpatient to this hospital":"long",
    "Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital":"long",
    "Left AMA":"left",
    "Expired":"exp",
    np.nan:"unk"}

In [177]:
for cat,name in discharge_disposition_map.items():
    if "discharge_("+name+")" in data.columns:
        data["discharge_("+name+")"]|=(data["discharge_disposition"]==cat).astype("int64")
    else:
        data["discharge_("+name+")"]=(data["discharge_disposition"]==cat).astype("int64")
data.drop("discharge_disposition",axis=1,inplace=True)
data=data.copy()

In [178]:
admission_source_map={
    " Emergency Room":"emerg",
    "Transfer from a hospital":"transf",
    " Transfer from another health care facility":"fed",
    " Transfer from a Skilled Nursing Facility (SNF)":"snf",
    " Transfer from hospital inpt/same fac reslt in a sep claim":"transf",
    " Transfer from critial access hospital":"crit",
    " Transfer from Ambulatory Surgery Center":"amb",
    " Physician Referral":"phys",
    "Clinic Referral":"clin",
    "HMO Referral":"hmo",
    np.nan:"unk"}

In [179]:
for cat,name in admission_source_map.items():
    if "adm_source_("+name+")" in data.columns:
        data["adm_source_("+name+")"]|=(data["admission_source"]==cat).astype("int64")
    else:
        data["adm_source_("+name+")"]=(data["admission_source"]==cat).astype("int64")
data.drop("admission_source",axis=1,inplace=True)
data=data.copy()

In [180]:
data.rename({"length_of_stay_in_hospital":"length_of_stay",
    "number_lab_tests":"#_lab",
    "non_lab_procedures":"#_non_lab",
    "number_of_medications":"#_meds"},
    axis=1,inplace=True)

In [181]:
for column in {"length_of_stay","#_lab","#_non_lab","#_meds"}:
    minimum=data[column].min()
    data[column]=(data[column]-minimum)/(data[column].max()-minimum)

In [182]:
def is_diabetes(code):
    if "." in code or code=="250":
        return 1
    else:
        return 0

In [183]:
data["diab1"]=(data["primary_diagnosis"].map(is_diabetes))
data["diab2"]=(data["secondary_diagnosis"].map(is_diabetes))
data["diab3"]=(data["additional_diagnosis"].map(is_diabetes))
data=data.copy()

In [184]:
def diabetes_type(code):
    if "." in code:
        return code.split(".")[1][0]
    else:
        return "?"

In [185]:
def controlled(code):
    if "." in code:
        suffix=code.split(".")[1]
        if len(suffix)==2:
            if suffix[1]=="0" or suffix[1]=="1":
                return "No"
            else:
                return "Yes"
        else:
            return "?"
    else:
        return "?"

In [186]:
for name,kind in {"diab1":"primary_diagnosis",
    "diab2":"secondary_diagnosis","diab3":"additional_diagnosis"}.items():
    data[name+"_type"]=(data[kind].map(diabetes_type)).astype("category")
    temporary_series=data[kind].map(controlled).astype("category")
    data[name+"_(controlled)"]=(temporary_series=="Yes").astype("int64")
    data[name+"_(uncontrolled)"]=(temporary_series=="No").astype("int64")
    for cat in [str(c) for c in range(10)]:
        data[name+"_type_("+cat+")"]=(data[name]==cat).astype("int64")
    data.drop(name+"_type",axis=1,inplace=True)
data=data.copy()

In [187]:
def icd9_conversion(code):
    if code=="?": return "?"
    if code[0]=="M": return "M"
    if code[0]=="V": return "V"
    if code[0]=="E": return "E"
    if "." in code: return "B"
    try:
        code=int(code)
        if 1<=code<140: return "I&PD"
        if 140<=code<240: return "NEO"
        if 240<=code<280: return "ENM&I"
        if 280<=code<290: return "B"
        if 290<=code<320: return "Men"
        if 320<=code<390: return "N"
        if 390<=code<460: return "C"
        if 460<=code<520: return "R"
        if 520<=code<580: return "D"
        if 580<=code<630: return "G"
        if 630<=code<680: return "PC&P"
        if 680<=code<710: return "S"
        if 710<=code<740: return "M&C"
        if 740<=code<760: return "Cong"
        if 760<=code<780: return "PP"
        if 780<=code<800: return "SS&Ill"
        if 800<=code<1000: return "I&P"
    except ValueError:
        return "?"
    return "?"

In [188]:
for kind in ["primary_diagnosis","secondary_diagnosis","additional_diagnosis"]:
    data[kind]=data[kind].map(icd9_conversion).astype("category")
for name,kind in {"diag1":"primary_diagnosis","diag2":"secondary_diagnosis",
    "diag3":"additional_diagnosis"}.items():
    for type,code in {"v":"V","e":"E","b":"B","ipd":"I&PD","neo":"NEO","enmi":"ENM&I","m":"M",
        "n":"N","c":"C","r":"R","d":"D","g":"G","pcp":"PC&P","s":"S","mc":"M&C","cong":"Cong",
        "pp":"PP","ip":"I&P","unk":"?"}.items():
        data[name+"_("+type+")"]=(data[kind]==code).astype("int64")
    data.drop(kind,axis=1,inplace=True)
data=data.copy()

In [189]:
data.rename({"number_diagnoses":"#_diag"},axis=1,inplace=True)

In [190]:
minimum=data["#_diag"].min()
data["#_diag"]=(data["#_diag"]-minimum)/(data["#_diag"].max()-minimum)

In [191]:
data["no_glucose_test"]=(data["glucose_test_result"]==np.nan).astype("int64")
glucose_map={np.nan:0,"Norm":0,">200":1,">300":1}
data["glucose"]=data["glucose_test_result"].replace(glucose_map).astype("int64")
data.drop("glucose_test_result",axis=1,inplace=True)
data=data.copy()

In [192]:
data["no_a1c_test"]=(data["a1c_test_result"]==np.nan).astype("int64")
a1c_map={np.nan:0,"Norm":0,">7":1,">8":1}
data["a1c"]=data["a1c_test_result"].replace(a1c_map).astype("int64")
data.drop("a1c_test_result",axis=1,inplace=True)
data=data.copy()

In [193]:
data["change_in_diab_meds"]=data["change_in_meds_during_hospitalization"].\
    replace({"Ch":1,"No":0})
data.drop("change_in_meds_during_hospitalization",axis=1,inplace=True)

In [194]:
data["prescribed_diabetes_meds"]=(data["prescribed_diabetes_meds"]=="Yes").astype("int64")

In [195]:
def count_medication(med):
    count=0
    for char in med:
        if char=="'":
            count+=1
    return count//2

In [196]:
data["#_of_diab_meds"]=data["medication"].map(count_medication)
minimum=data["#_of_diab_meds"].min()
data["#_of_diab_meds"]=(data["#_of_diab_meds"]-minimum)/(data["#_of_diab_meds"].max()-minimum)

In [197]:
import re
unique_medications=set()
for meds in data["medication"]:
    for compound_med in (re.search(r'(?<=\[)(.*?)(?=\])',meds)[0]).split(sep=','):
        if compound_med!="":
            compound_med=compound_med.strip()
            for med in compound_med.split(sep='-'):
                unique_medications.add(med.strip("'"))
for med in unique_medications:
    data[med]=data["medication"].map(lambda meds: med in meds).astype("int64")
data.drop("medication",axis=1,inplace=True)

In [198]:
data["binary_target"]=(data["readmitted_binary"]=="Yes").astype("int64")
data.drop("readmitted_binary",axis=1,inplace=True)
binary_target=data["binary_target"].values.astype(np.int64)
data.drop("binary_target",axis=1,inplace=True)

In [199]:
data["multiclass_target_(<30)"]=(data["readmitted_multiclass"]=="<30 days").astype("int64")
data["multiclass_target_(>30)"]=(data["readmitted_multiclass"]==">30 days").astype("int64")
data.drop("readmitted_multiclass",axis=1,inplace=True)
multiclass_target=data[["multiclass_target_(<30)","multiclass_target_(>30)"]].values.astype(np.int64)
data.drop("multiclass_target_(<30)",axis=1,inplace=True)
data.drop("multiclass_target_(>30)",axis=1,inplace=True)

In [200]:
features=sorted(data.columns)
data=data[features].copy()

In [201]:
training_mask=(data.std(axis=0)==0.0).values

In [202]:
for column in data.columns[training_mask]:
    data.drop(column,axis=1,inplace=True)
training_mask=~training_mask
training_features=data.columns

In [203]:
from sklearn.neighbors import KNeighborsClassifier
target=data["age"]
nn_data=data[list(set(training_features)-set({"age"}))]
nn_mask=target.isna()
knn=KNeighborsClassifier(n_neighbors=5,weights="distance",p=1)
knn.fit(nn_data[~nn_mask],target[~nn_mask])
data.loc[nn_mask,"age"]=knn.predict(nn_data[nn_mask])
print(data["age"].isna().any())
data["age"]=data["age"].astype("int64")
data["age"]=(data["age"]-data["age"].min())/(data["age"].max()-data["age"].min())

False


In [204]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71220 entries, 533253 to 459757
Columns: 149 entries, #_diag to troglitazone
dtypes: float64(11), int64(138)
memory usage: 81.5 MB


In [206]:
from scipy.stats import norm
for column in data.columns[data.dtypes.eq("float64")]:
    data[column]=norm.ppf((data[column].copy().rank().values-0.5)/data[column].count())
    data[column]=(data[column]-data[column].min())/(data[column].max()-data[column].min())

In [149]:
from sklearn.feature_selection import chi2
p_values=chi2(data.values.astype(np.float64),binary_target)[1]
mask=(p_values>=0.0001)

---

In [150]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
array=data.values.astype(np.float64)#[:,mask]
folds=4
cross_validator=StratifiedKFold(n_splits=folds,shuffle=True,random_state=0)
prediction=np.zeros(len(binary_target))
training_acuracy,training_f1=0.0,0.0
for fold,(train_index,val_index) in enumerate(cross_validator.split(array,binary_target)):
    model=RandomForestClassifier(class_weight="balanced",n_estimators=128,
        min_samples_split=256,random_state=0,min_impurity_decrease=10**(-6),max_features=32)
    model.fit(array[train_index],binary_target[train_index])
    train_preds=model.predict(array[train_index])
    training_acuracy+=accuracy_score(binary_target[train_index],train_preds)/folds
    training_f1+=f1_score(binary_target[train_index],train_preds)/folds
    prediction[val_index]=model.predict(array[val_index])
print("\nTraining:")
print(" Accuracy:  ",training_acuracy.round(3))
print(" F1:  ",training_f1.round(4))
print("\nValidation:")
print(" Accuracy:  ",accuracy_score(binary_target,prediction).round(3))
print(" F1:  ",f1_score(binary_target,prediction).round(4))
print(" Confusion:\n",confusion_matrix(binary_target,prediction))


Training:
 Accuracy:   0.77
 F1:   0.3914

Validation:
 Accuracy:   0.733
 F1:   0.2861
 Confusion:
 [[48432 14841]
 [ 4143  3804]]


In [868]:
from sklearn.ensemble import RandomForestClassifier
array=data.values.astype(np.float64)
model=RandomForestClassifier(class_weight="balanced",n_estimators=128,
    min_samples_split=256,random_state=0,min_impurity_decrease=10**(-6),max_features=32)
model.fit(array,binary_target)

RandomForestClassifier(class_weight='balanced', max_features=32,
                       min_impurity_decrease=1e-06, min_samples_split=256,
                       n_estimators=128, random_state=0)

---

In [197]:
test_data=pd.read_csv("./project/test.csv")
test_data["id"]=test_data["encounter_id"]
test_data.drop("encounter_id",axis=1,inplace=True)
test_data.drop("patient_id",axis=1,inplace=True)
test_data.drop("country",axis=1,inplace=True)
test_data.drop("average_pulse_bpm",axis=1,inplace=True)
test_data.set_index("id",inplace=True)

In [198]:
data["weight"]=data["weight"].map(lambda x: 0 if x=="?" else 1)

In [199]:
for cat,name in race_map.items():
    test_data["race_("+name+")"]=(test_data["race"]==cat).astype("int64")
test_data.drop("race",axis=1,inplace=True)
test_data=test_data.copy()

In [200]:
test_data["gender"]=test_data["gender"].map(lambda g: 1 if g=="Female" else 0)

In [201]:
test_data["age"]=test_data["age"].map(lambda age: int(age[1]) if not age is np.nan else np.nan).astype("Int64")

In [202]:
for cat,name in payer_code_map.items():
    test_data["code_("+name+")"]=(test_data["payer_code"]==cat).astype("int64")
test_data.drop("payer_code",axis=1,inplace=True)
test_data=test_data.copy()

In [203]:
test_data.rename({"outpatient_visits_in_previous_year":"#_outpatient",
    "emergency_visits_in_previous_year":"#_emergency",
    "inpatient_visits_in_previous_year":"#_inpatient"},
    axis=1,inplace=True)

In [204]:
test_data["#_of_visits"]=test_data["#_outpatient"]+test_data["#_inpatient"]+test_data["#_emergency"]

In [205]:
for column in {"#_of_visits","#_outpatient","#_inpatient","#_emergency"}:
    test_data[column]=test_data[column].map(lambda value: np.log1p(value))
    minimum=test_data[column].min()
    test_data[column]=(test_data[column]-minimum)/(test_data[column].max()-minimum)

In [206]:
for cat,name in admission_type_map.items():
    if "adm_type_("+name+")" in test_data.columns:
        test_data["adm_type_("+name+")"]|=(test_data["admission_type"]==cat).astype("int64")
    else:
        test_data["adm_type_("+name+")"]=(test_data["admission_type"]==cat).astype("int64")
test_data.drop("admission_type",axis=1,inplace=True)
test_data=test_data.copy()

In [207]:
for cat,name in medical_specialty_map.items():
    if "med_spec_("+name+")" in test_data.columns:
        test_data["med_spec_("+name+")"]|=(test_data["medical_specialty"]==cat).astype("int64")
    else:
        test_data["med_spec_("+name+")"]=(test_data["medical_specialty"]==cat).astype("int64")
test_data.drop("medical_specialty",axis=1,inplace=True)
test_data=test_data.copy()

In [208]:
for cat,name in discharge_disposition_map.items():
    if "discharge_("+name+")" in test_data.columns:
        test_data["discharge_("+name+")"]|=(test_data["discharge_disposition"]==cat).astype("int64")
    else:
        test_data["discharge_("+name+")"]=(test_data["discharge_disposition"]==cat).astype("int64")
test_data.drop("discharge_disposition",axis=1,inplace=True)
test_data=test_data.copy()

In [209]:
for cat,name in admission_source_map.items():
    if "adm_source_("+name+")" in test_data.columns:
        test_data["adm_source_("+name+")"]|=(test_data["admission_source"]==cat).astype("int64")
    else:
        test_data["adm_source_("+name+")"]=(test_data["admission_source"]==cat).astype("int64")
test_data.drop("admission_source",axis=1,inplace=True)
test_data=test_data.copy()

In [210]:
test_data.rename({"length_of_stay_in_hospital":"length_of_stay",
    "number_lab_tests":"#_lab",
    "non_lab_procedures":"#_non_lab",
    "number_of_medications":"#_meds"},
    axis=1,inplace=True)

In [211]:
for column in {"length_of_stay","#_lab","#_non_lab","#_meds"}:
    minimum=test_data[column].min()
    test_data[column]=(test_data[column]-minimum)/(test_data[column].max()-minimum)

In [212]:
test_data["diab1"]=(test_data["primary_diagnosis"].map(is_diabetes))
test_data["diab2"]=(test_data["secondary_diagnosis"].map(is_diabetes))
test_data["diab3"]=(test_data["additional_diagnosis"].map(is_diabetes))
test_data=test_data.copy()

In [213]:
for name,kind in {"diab1":"primary_diagnosis",
    "diab2":"secondary_diagnosis","diab3":"additional_diagnosis"}.items():
    test_data[name+"_type"]=(test_data[kind].map(diabetes_type)).astype("category")
    temporary_series=test_data[kind].map(controlled).astype("category")
    test_data[name+"_(controlled)"]=(temporary_series=="Yes").astype("int64")
    test_data[name+"_(uncontrolled)"]=(temporary_series=="No").astype("int64")
    for cat in [str(c) for c in range(10)]:
        test_data[name+"_type_("+cat+")"]=(test_data[name]==cat).astype("int64")
    test_data.drop(name+"_type",axis=1,inplace=True)
test_data=test_data.copy()

In [214]:
for kind in ["primary_diagnosis","secondary_diagnosis","additional_diagnosis"]:
    test_data[kind]=test_data[kind].map(icd9_conversion).astype("category")
for name,kind in {"diag1":"primary_diagnosis","diag2":"secondary_diagnosis",
    "diag3":"additional_diagnosis"}.items():
    for type,code in {"v":"V","e":"E","b":"B","ipd":"I&PD","neo":"NEO","enmi":"ENM&I","m":"M",
        "n":"N","c":"C","r":"R","d":"D","g":"G","pcp":"PC&P","s":"S","mc":"M&C","cong":"Cong",
        "pp":"PP","ip":"I&P","unk":"?"}.items():
        test_data[name+"_("+type+")"]=(test_data[kind]==code).astype("int64")
    test_data.drop(kind,axis=1,inplace=True)
test_data=test_data.copy()

In [215]:
test_data.rename({"number_diagnoses":"#_diag"},axis=1,inplace=True)
minimum=test_data["#_diag"].min()
test_data["#_diag"]=(test_data["#_diag"]-minimum)/(test_data["#_diag"].max()-minimum)

In [216]:
test_data["no_glucose_test"]=(test_data["glucose_test_result"]==np.nan).astype("int64")
glucose_map={np.nan:0,"Norm":0,">200":1,">300":1}
test_data["glucose"]=test_data["glucose_test_result"].replace(glucose_map).astype("int64")
test_data.drop("glucose_test_result",axis=1,inplace=True)
test_data=test_data.copy()

In [217]:
test_data["no_a1c_test"]=(test_data["a1c_test_result"]==np.nan).astype("int64")
a1c_map={np.nan:0,"Norm":0,">7":1,">8":1}
test_data["a1c"]=test_data["a1c_test_result"].replace(a1c_map).astype("int64")
test_data.drop("a1c_test_result",axis=1,inplace=True)
test_data=test_data.copy()

In [218]:
test_data["change_in_diab_meds"]=test_data["change_in_meds_during_hospitalization"].\
    replace({"Ch":1,"No":0})
test_data.drop("change_in_meds_during_hospitalization",axis=1,inplace=True)

In [219]:
test_data["prescribed_diabetes_meds"]=(test_data["prescribed_diabetes_meds"]=="Yes").astype("int64")

In [220]:
test_data["#_of_diab_meds"]=test_data["medication"].map(count_medication)
minimum=test_data["#_of_diab_meds"].min()
test_data["#_of_diab_meds"]=(test_data["#_of_diab_meds"]-minimum)/(test_data["#_of_diab_meds"].max()-minimum)

In [221]:
for med in unique_medications:
    test_data[med]=test_data["medication"].map(lambda meds: med in meds).astype("int64")
test_data.drop("medication",axis=1,inplace=True)

In [222]:
test_data=test_data[training_features].copy()

KeyError: "['patient_id'] not in index"

In [74]:
from sklearn.neighbors import KNeighborsClassifier
target=test_data["age"]
nn_data=test_data[list(set(training_features)-set({"age"}))]
nn_mask=target.isna()
knn=KNeighborsClassifier(n_neighbors=5,weights="distance",p=1)
knn.fit(nn_data[~nn_mask],target[~nn_mask])
test_data.loc[nn_mask,"age"]=knn.predict(nn_data[nn_mask])
print(test_data["age"].isna().any())
test_data["age"]=test_data["age"].astype("int64")
test_data["age"]=(test_data["age"]-test_data["age"].min())/(test_data["age"].max()-test_data["age"].min())

False


In [75]:
from scipy.stats import norm
for column in test_data.columns[test_data.dtypes.eq("float64")]:
    test_data[column]=norm.ppf((test_data[column].copy().rank().values-0.5)/test_data[column].count())
    test_data[column]=(test_data[column]-test_data[column].min())/(test_data[column].max()-test_data[column].min())

In [76]:
test_array=test_data.values.astype(np.float64)

In [82]:
test_predictions=model.predict(test_array)
test_predictions=pd.DataFrame(data={"readmitted_binary":test_predictions},
    index=test_data.index)
test_predictions["readmitted_binary"]=test_predictions["readmitted_binary"].replace({0:"No",1:"Yes"}).astype("object")
test_predictions.to_csv(path_or_buf="./first_submission.csv",index_label="encounter_id")